In [1]:
import numpy as np
import pandas as pd
import pyodbc as pyo
import sqlalchemy as sa
import numpy as np
import math
import datetime
connection_uri = (
    "mssql+pyodbc://TEDPHO0134\TANDT_DP/DP_Tan?driver=ODBC+Driver+17+for+SQL+Server"
)
conn = sa.create_engine(connection_uri)

<h4>1. Lấy bảng Commision <h4>

In [2]:
cutoff = '202401'
commission_query = f"select * from AGENT_COM_CUTOFF where CUTOFF like '{cutoff}%'"
agent_com = pd.read_sql(commission_query, conn)

In [3]:
#1.1 Lấy bảng AGENT_INFO 
commission_query = f"select * from AGENT_INFO where CUTOFF like '{cutoff}%'"
agent_info = pd.read_sql(commission_query, conn)

<h4>2. Lấy bảng PY2 <h4>

In [9]:
wcc2_query = f"select * from AGENT_COM_CUTOFF where CUTOFF like '{cutoff}%'"
py2 = pd.read_sql(py2_query, conn)

<h4>3. Copy bảng <h4>

In [4]:
com_table = agent_com
#py2_table = py2

<h4> 4. Xét Period <h4>

In [56]:
Period_Com = com_table

In [57]:
#LƯU Ý CÁCH CỘNG 1 NGÀY VÀO CỘT APPLIED PREMIUM DATE
Period_Com ['Date_Period_1'] = np.where( (Period_Com['Policy Status'] == 'FL'), Period_Com['Issued Date'],
np.where( (Period_Com['Receive Policy date'].isnull()), Period_Com['Issued Date'],
np.where((((Period_Com['Applied Premium Date'] + datetime.timedelta(days=1)) - Period_Com['Receive Policy date']) / np.timedelta64(1, 'D')) < 22,
Period_Com['Issued Date'], Period_Com['Applied Premium Date'])))

In [58]:
#Logic: Nếu ngày Period đầu tiên xét đi trước ngày ACK, thì lấy ngày ACK để xét trong kỳ tính phí
Period_Com ['Date_Period'] = np.where(
(((Period_Com['Receive Policy date']) - Period_Com['Date_Period_1']) / np.timedelta64(1, 'D')) >0,
Period_Com['Receive Policy date'], Period_Com['Date_Period_1']
)

<h4> 5. Xét các cột ngày theo CUTOFF <h4>

In [59]:
#Thêm cột ngày xét
Monthly_Com = Period_Com
Monthly_Com['Date_On_Set'] = "" #Tạo ra cột trống
Monthly_Com['Date_On_Set']= Monthly_Com['Date_On_Set'].str.cat(Monthly_Com['CUTOFF'].str[:4] ,sep ="")
Monthly_Com['Date_On_Set']= Monthly_Com['Date_On_Set'].str.cat(Monthly_Com['CUTOFF'].str[4:6] ,sep ="-")
Monthly_Com['Date_On_Set']= Monthly_Com['Date_On_Set'].str.cat(Monthly_Com['CUTOFF'].str[6:8] ,sep ="-")
#Thêm cột ưu tiên
Monthly_Com['Priority'] = Monthly_Com['CUTOFF'].rank(method='dense', ascending=True)
#Phải biến thành kiểu datetime trước
Monthly_Com['Date_On_Set'] = pd.to_datetime(Monthly_Com['Date_On_Set'])
#Thêm cột ngày đầu tháng:
#-> Đưa vào ngày 1 của tháng
if not('20240229' in Monthly_Com['CUTOFF'].values): 
    Monthly_Com['Start_Month'] = (Monthly_Com['Date_On_Set'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))
else:
    Monthly_Com.loc[:,'Start_Month'] = '2024-01-27'

<h4> 6. Xét các điều kiện <h4>

In [60]:
calculate_condition = ((Monthly_Com['Date_Period'] >= Monthly_Com['Start_Month']) & (Monthly_Com['Date_Period'] <= Monthly_Com['Date_On_Set'])) |\
((Monthly_Com['Applied Premium Date'] >= Monthly_Com['Start_Month']) & (Monthly_Com['Applied Premium Date'] <= Monthly_Com['Date_On_Set']))
( (Monthly_Com['Policy Status'] != 'SU') | (Monthly_Com['Policy Status'] != 'TR'))
calculate_month = Monthly_Com.loc[calculate_condition]
calculate_month

,Policy No,Product Code,Premium transaction,Premium transaction name,Premium Collected,Collected Date,Applied Premium Date,FYC,RYC,Shared Commission Rate,...,RISK_COMMENCE_DATE,Receive Policy date,Premium Year,CUTOFF,RETURN_DATE,Date_Period_1,Date_Period,Date_On_Set,Priority,Start_Month
316,80001330,EN04,B522,Premium Collection,7449000.0,2024-01-08,2024-01-08,0.0,149000.0,100.0,...,2015-12-21,2015-12-26,9,20240115,None,2024-01-08,2024-01-08,2024-01-15,1.0,2024-01-01
702,80001331,EN04,B522,Premium Collection,7523000.0,2024-01-08,2024-01-08,0.0,150000.0,100.0,...,2015-12-21,2015-12-26,9,20240115,None,2024-01-08,2024-01-08,2024-01-15,1.0,2024-01-01
5604,80001378,EN04,B522,Premium Collection,11328000.0,2024-01-04,2024-01-04,0.0,227000.0,100.0,...,2015-12-27,2016-01-07,9,20240115,None,2024-01-04,2024-01-04,2024-01-15,1.0,2024-01-01
11179,80002454,EE02,B522,Premium Collection,16417000.0,2024-01-02,2024-01-02,0.0,328000.0,100.0,...,2016-03-30,2016-04-11,8,20240115,None,2024-01-02,2024-01-02,2024-01-15,1.0,2024-01-01
11180,80002454,RTR2,B522,Premium Collection,406000.0,2024-01-02,2024-01-02,0.0,8000.0,100.0,...,2016-03-30,2016-04-11,8,20240115,None,2024-01-02,2024-01-02,2024-01-15,1.0,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172045,80123950,RHB1,B522,Premium Collection,259000.0,2024-01-26,2024-01-26,0.0,26000.0,100.0,...,2022-10-27,2022-11-02,2,20240126,None,2024-01-26,2024-01-26,2024-01-26,2.0,2024-01-01
3172046,80123950,RHS1,B522,Premium Collection,85000.0,2024-01-26,2024-01-26,0.0,17000.0,100.0,...,2022-10-27,2022-11-02,2,20240126,None,2024-01-26,2024-01-26,2024-01-26,2.0,2024-01-01
3172047,80123950,URA5,B522,Premium Collection,448000.0,2024-01-26,2024-01-26,0.0,31000.0,100.0,...,2022-10-27,2022-11-02,2,20240126,None,2024-01-26,2024-01-26,2024-01-26,2.0,2024-01-01
3172048,80123950,URC6,B522,Premium Collection,329000.0,2024-01-26,2024-01-26,0.0,23000.0,100.0,...,2022-10-27,2022-11-02,2,20240126,None,2024-01-26,2024-01-26,2024-01-26,2.0,2024-01-01


<h4>7. Join với Agent Info để lấy Terminate Date <h4>

In [62]:
calculate_month = pd.merge(calculate_month, agent_info, left_on = ['Commission Agent', 'CUTOFF'], 
                           right_on = ['Agent_Number', 'CUTOFF'], how = 'left' )

In [63]:
#Bốc ra các cột cần thiết
calculate_month = calculate_month[['Date_Period', 'Commission Agent', 'Policy No',
       'Premium transaction name', 'Policy Status', 'Applied Premium Date', 'Issued Date_x', 'Receive Policy date', 'Freelook',
       'CUTOFF', 'Premium Collected', 'FYC', 'RYC', 'Terminated_date']].rename(columns={'Issued Date_x': 'Issued Date'})
                                                                                        

In [64]:
#Tạo ra cột Status với các điều kiện
#1. Chưa pass FL
#2. Chưa có ACK
#3. Không pass FL trong 21 ngày
#4. Ter trước khi phát sinh phí
#5. Thanh toán
#len của CheckTer >1 => Terminated
calculate_month['Status'] = np.where((calculate_month['Freelook']<=21) & (calculate_month['Receive Policy date'].notnull()), "1", 
                     np.where(calculate_month['Receive Policy date'].isnull(), "2",
                     np.where((calculate_month['Policy Status'] == "FL") & (calculate_month['Terminated_date'].isnull() ), "3",
                     np.where((calculate_month['Policy Status'] != "FL") & calculate_month['Terminated_date'].notnull(), "4", 
                     np.where((calculate_month['Policy Status'] != "FL") & (calculate_month['Terminated_date'].isnull()) & (calculate_month['Freelook']>21), "5","")     
                             ))))

In [86]:
Filter_Month = calculate_month[['Date_Period','Commission Agent', 'Terminated_date' ,'Policy No', 'Premium transaction name', 'Policy Status', 'Applied Premium Date',
 'Issued Date', 'Receive Policy date', 'Freelook' ,'CUTOFF', 'Premium Collected', 'FYC', 'RYC', 'Status']]
Filter_Month['Mark_Time'] =  Filter_Month.groupby(['Policy No', 'Applied Premium Date', 'Premium Collected', 'Status'])['CUTOFF'].rank(method="dense", ascending=True)
#Lấy các dòng cần giữ 2 dòng
keep_two = Filter_Month[(Filter_Month['Status'] == "1") | (Filter_Month['Status'] == "2") ]
#Lấy các dòng cần giữ 1 dòng
keep_one = Filter_Month[(Filter_Month['Status'] == "3") | (Filter_Month['Status'] == "4") | (Filter_Month['Status'] == "5") ]
keep_one = keep_one[keep_one['Mark_Time'] == 1]
full_modified = pd.concat([keep_one, keep_two])

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_22036\2521604217.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Filter_Month['Mark_Time'] =  Filter_Month.groupby(['Policy No', 'Applied Premium Date', 'Premium Collected', 'Status'])['CUTOFF'].rank(method="dense", ascending=True)


In [87]:
full_modified = full_modified[['Date_Period','Commission Agent', 'Terminated_date','Policy No', 'Premium transaction name',
       'Policy Status', 'Applied Premium Date', 'Issued Date', 'Freelook',
       'Receive Policy date', 'CUTOFF', 'Premium Collected', 'FYC', 'RYC', 'Mark_Time',
       'Status']]
#Rút ra cột Final FYC để về 0 trong các trạng thái không thanh toán
full_modified.loc[(full_modified['Status'].isin(['1','2','4'])), 'Final_FYC'] = "0"
full_modified.loc[(full_modified['Status'].isin(['3','5'])), 'Final_FYC'] = full_modified['FYC']   

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_22036\2424443321.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  full_modified.loc[(full_modified['Status'].isin(['1','2','4'])), 'Final_FYC'] = "0"


In [88]:
#Rút ra cột Final RYC để về 0 trong các trạng thái không thanh toán
full_modified.loc[(full_modified['Status'].isin(['1','2','4'])), 'Final_RYC'] = "0"
full_modified.loc[(full_modified['Status'].isin(['3','5'])), 'Final_RYC'] = full_modified['RYC'] 

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_22036\4155379092.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  full_modified.loc[(full_modified['Status'].isin(['1','2','4'])), 'Final_RYC'] = "0"


In [89]:
#Tính Total Commision
full_modified['Total_Commision'] = full_modified['Final_FYC'] + full_modified['Final_RYC']

<h4>7. Xét WCC <h4>

In [69]:
###### Chỉ xét WCC trên Policy
wcc_policy_query = f"select * from WCC_Policy where CUTOFF like '{cutoff}%'"
wcc_policy = pd.read_sql(wcc_policy_query, conn)
policy_wcc = wcc_policy

In [90]:
#Xét điều kiên bảng PolicyWCC
policy_wcc.loc[(policy_wcc['finalResult'] == "WCC không thành công"), "Policy_Result"] = "Fail"
policy_wcc.loc[(policy_wcc['finalResult'] != "WCC không thành công"), "Policy_Result"] = "Success"

In [91]:
# Đổi sang kiểu int để join bảng
policy_wcc['CONTRACT_NUMBER'] = policy_wcc['CONTRACT_NUMBER'].astype('int')
full_modified['Policy No'] = full_modified['Policy No'].astype('int')

In [92]:
Full_Com = pd.merge(full_modified, policy_wcc, left_on = 'Policy No', right_on = 'CONTRACT_NUMBER' ,how = 'left')

In [93]:
Full_Com = Full_Com[['Date_Period', 'Commission Agent', 'Policy No',
       'Premium transaction name', 'Policy Status', 'Applied Premium Date',
       'Issued Date', 'Receive Policy date','Freelook', 'CUTOFF_x', 'Premium Collected',
       'FYC', 'RYC', 'Mark_Time', 'Status', 'Final_FYC', 'Final_RYC',
       'Total_Commision', 'Policy_Result']].rename(columns={"CUTOFF_x": "CUTOFF"})

In [94]:
Full_Com['Final_Result'] = np.where((Full_Com['Policy_Result'] == 'Success'), Full_Com['Status'],
np.where((Full_Com['Policy_Result'] == 'Fail') & (Full_Com['Status'] == '5'), "6",Full_Com['Status']))

<h4> SUM Group by kết quả <h4>

In [106]:
Groupby_Com = Full_Com.groupby(['Commission Agent', 'Policy No', 'CUTOFF', 'Applied Premium Date', 'Issued Date'
      ])[['Premium Collected','FYC','RYC', 'Final_FYC', 'Final_RYC',
       'Total_Commision']].sum()

In [107]:
Groupby_Com = Groupby_Com.reset_index().rename(columns={'Commission Agent': 'AgentNumber', 'Policy No': 'PolicyNo', 'CUTOFF': 'CUT_OFF'})

In [108]:
Groupby_Com = pd.merge(Groupby_Com, Full_Com[['Commission Agent', 'Policy No' ,'Freelook', 'CUTOFF', 'Status', 'Final_Result']],
    left_on = ['AgentNumber', 'PolicyNo', 'CUT_OFF'], right_on =['Commission Agent', 'Policy No', 'CUTOFF'], how = 'left')

In [109]:
#Loại bỏ dòng trùng
Groupby_Com = Groupby_Com.drop_duplicates()

In [110]:
Groupby_Com['FYC/RYC'] = np.where(Groupby_Com['FYC'] !=0, 'FYC', 
                         np.where(Groupby_Com['RYC'] !=0, 'RYC','' ))

In [111]:
#Xét các kỳ thanh toán:
Groupby_Com['KyTinhToan'] = Groupby_Com['CUTOFF']
Groupby_Com['KyKhoaSo'] = Groupby_Com['CUTOFF']
Groupby_Com['KyThanhToan'] = np.where(Groupby_Com['Status'].isin(['3','5']), Groupby_Com['CUTOFF'], "")

In [112]:
Groupby_Com = Groupby_Com[['AgentNumber', 'PolicyNo', 'CUT_OFF', 'Applied Premium Date', 'Issued Date', 'Premium Collected',  'Freelook', 'FYC',
 'RYC', 'Final_FYC', 'Final_RYC', 'Total_Commision',  'FYC/RYC',  'Status',  'Final_Result', 
 'KyTinhToan', 'KyKhoaSo',  'KyThanhToan']]

In [113]:
Final_Result = pd.merge(Groupby_Com, Full_Com[['Policy No', 'Receive Policy date', 'CUTOFF']], left_on = ['PolicyNo', 'CUT_OFF'],
                       right_on = ['Policy No', 'CUTOFF'], how = 'left')

In [114]:
Final_Result = Final_Result[['AgentNumber', 'PolicyNo', 'CUT_OFF', 'Applied Premium Date', 'Issued Date', 'Premium Collected', 'Receive Policy date' ,'Freelook', 
                             'FYC', 'RYC', 'Final_FYC', 'Final_RYC', 'Total_Commision',  'FYC/RYC',  'Status',  'Final_Result', 
 'KyTinhToan', 'KyKhoaSo',  'KyThanhToan']]

In [115]:
Final_Result = Final_Result.drop_duplicates()

In [116]:
Final_Result.to_csv('check_new.csv')